In [ ]:
import json

# load everything from json, at least 2GB of ram is needed for this
game = {}
# all keys inside game params
keys = {}
with open('tools/GameParams.json', 'r') as j:
    game = json.load(j)
    keys = game.keys()
    # just to make sure something is printed and everything are read
    print(len(keys))

ships = {}
with open('tools/ship_all.json', 'r') as j:
    ships = json.load(j)
    print(len(ships))


In [ ]:
# write json object to disk
def writeToJson(json_dict, name):
    with open(name, 'w+') as l:
        l.write(json.dumps(json_dict))

In [ ]:
# find keys in object, can be improved to go deep
def findKey(name, keys):
    temp = []
    for key in keys:
        if name.lower() in key.lower():
            temp.append(key)
    return temp

In [ ]:
# Find unique update and their ship
unique_upgrades = findKey('Special_Mod', keys)
# this adds ship id to legendary upgrade
legendary = {}
# this add legendary upgrades to ship
ship_upgrade = {}
# get their id and the ship they belong to
for upgrade in unique_upgrades:
    temp = game[upgrade]
    upgrade_id = temp['id']
    upgrade_id_str = str(upgrade_id)
    # remember to add slot later
    slot = temp['slot']
    
    legendary[upgrade_id_str] = []
    for ship in temp['ships']:
        ship_id = game[ship]['id']
        ship_id_str = str(ship_id)
        
        # init
        if not ship_id_str in ship:
            ship_upgrade[ship_id_str] = {"legendary_upgrades": []}

        # you need to the slot as well
        ship_upgrade[ship_id_str]["legendary_upgrades"].append({"upgrade_id": upgrade_id_str, "slot": slot})
        legendary[upgrade_id_str].append({"ship_id": ship_id, "slot": slot})

writeToJson(legendary, 'legendary_upgrade.json')
writeToJson(ship_upgrade, 'ship_legendary_upgrade.json')


In [ ]:
findKey('PASB518', keys)

In [ ]:
writeToJson(game['PIPA041_203MM_55_1924_CS'], 'venezia_sap.json')

In [ ]:
game['PCEM004_Halloween19_10lvl']

In [ ]:
def retrieveDataFromShip(ship):
    temp = {}
    permoflages = ship['permoflages']
    nativePermoflage = ship['nativePermoflage']
    permoflages.append(nativePermoflage)
    # we need to get all permoflage_id
    permo_ids = []
    for permo in permoflages:
        if permo != '':
            permo_ids.append(game[permo]['id'])
    # add it to temp
    temp['permoflages'] = permo_ids
    return temp

In [ ]:
# loop through all ship 
additional_ship_info = {}
for ship in ships:
    ship_str = ships[ship]['ship_id_str']
    # it should only has one
    additional_ship_info[ship] = retrieveDataFromShip(game[findKey(ship_str, keys)[0]])
writeToJson(additional_ship_info, 'ship_additional.json')
